<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to _n_cols_ since we will need this number when building our network.


In [9]:
n_cols = predictors.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


#### Let's go ahead and import the Keras library


In [10]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [11]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layer of 10 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [13]:
# build the model
model = regression_model()

Next, we will train the model using the _fit_ method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.
The training is repeated 


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

mse_list = []

for i in range(50):
    # random split data in train and test set
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)
    
    # fit the model
    model.fit(X_train, y_train, epochs=50, verbose=0)
    
    # predict and evaluate the model
    y_pred = model.predict(X_test, verbose=0)

    mse = mean_squared_error(y_test, y_pred)
    print(f"iteration: {i}, mse: {mse}")

    mse_list.append(mse)

iteration: 0, mse: 147.94227129995224
iteration: 1, mse: 117.73867723874542
iteration: 2, mse: 110.66307127422556
iteration: 3, mse: 133.60807528983463
iteration: 4, mse: 109.5584886393739
iteration: 5, mse: 110.60958276842949
iteration: 6, mse: 115.25413977882481
iteration: 7, mse: 108.66416735463962
iteration: 8, mse: 110.51560440562797
iteration: 9, mse: 113.13605315315391
iteration: 10, mse: 111.75896340743704
iteration: 11, mse: 111.25649522209622
iteration: 12, mse: 114.88149570280648
iteration: 13, mse: 108.40454818222285
iteration: 14, mse: 113.0115283873646
iteration: 15, mse: 109.63091487327523
iteration: 16, mse: 109.99829341496181
iteration: 17, mse: 157.4093910598439
iteration: 18, mse: 118.96043964019184
iteration: 19, mse: 108.91161162724697
iteration: 20, mse: 108.99145046181297
iteration: 21, mse: 115.85678679437534
iteration: 22, mse: 116.27482063838157
iteration: 23, mse: 110.16612806054255
iteration: 24, mse: 110.74377194719989
iteration: 25, mse: 115.5710017293829


Use pandas to compute standard deviation and mean of the mean squared errors

In [15]:
mse_df = pd.DataFrame(mse_list)
std_dev = mse_df.std()
mean = mse_df.mean()

print(f"standard deviavion: {std_dev[0]}, mean: {mean[0]}")

standard deviavion: 31.25107971362407, mean: 94.35771822650895
